# Here a kick scan from the data from the fire-brigate department

In [73]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
from math import radians, cos, sin, asin, sqrt,atan2

## Call the data sets

In [61]:
df_in = pd.read_csv('data\JADS\incidenten_2017.csv', sep=';', decimal=',')
df_dep = pd.read_csv('data\JADS\inzetten_2017.csv', sep=';', decimal=',')
locations = pd.read_excel('data\JADS\kazernepositie en voertuigen.xlsx', sheet_name='adressen')

C:\Users\s157084\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#(in) incident starts = dim_incident_start_datumtijd
#(in) incident finish = dim_incident_eind_datumtijd
#(dep) alarm is activated = inzet_gealarmeerd_datumtijd
#(dep) leave = inzet_uitgerukt_datumtijd
#(dep) arrive = inzet_terplaatse_datumtijd


In [62]:
locations

,kazerne,adres,postcode,plaats,lon,lat,dispatch (seconds)
0,AALSMEER,Zwarteweg 77,1431 VJ,Aalsmeer,4.764433,52.259274,196.0
1,AMSTELVEEN,Oranjebaan 1,1183 NN,Amstelveen,4.881831,52.301700,196.0
2,ANTON,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0
3,DIEMEN,D.J. den Hartoglaan 2,1111 ZC,Diemen,4.964767,52.341137,392.0
4,DIRK,Honthorststraat 27,1071 DC,Amsterdam,4.885874,52.357891,187.0
5,DRIEMOND,Lentestraat 30,1109 AT,Driemond,5.019351,52.306971,NaN
6,DUIVENDRECHT,Rijksstraatweg 58,1115 AT,Duivendrecht,4.940402,52.332296,274.0
7,HENDRIK,Marnixstraat 170,1016 TG,Amsterdam,4.875658,52.372561,193.0
8,IJSBRAND,Displaystraat 1,1033 RZ,Amsterdam,4.889573,52.406305,191.0
9,LANDELIJK NOORD,Dorpsweg 33,1090 AD,Amsterdam,4.994345,52.392532,485.0


In [87]:
df_in.T

,0,1,2,3,4,5,6,7,8,9,...,12744,12745,12746,12747,12748,12749,12750,12751,12752,12753
dim_incident_id,269907,270291,270421,268697,268266,268088,268101,267569,267387,267039,...,295024,294938,294943,295001,294973,294792,295281,295276,295277,295278
dim_incident_straatnaam_nen,Amstelveenseweg,De Wittenkade,Commelinstraat,Lindengracht,Lauriergracht,Elzenstraat,Deymanstraat,Zeilstraat,Eger,Sijbrandakker,...,Jacob van Lennepplnts,Leeuwendalersweg,Grianestraat,Talmastraat,Oostelijke Handelskade,Loenermark,Oosterlengte,Descartesstraat,Couperuslaan,Herenstraat
dim_incident_postcode,1076CR,1052DC,1093TK,1015KJ,1016RL,1032EN,1091SE,1075SH,1423GE,1541VX,...,1187AL,1055JE,1055EZ,1182EK,1019BN,1025VL,1034KZ,NaN,NaN,NaN
dim_incident_plaatsnaam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Uithoorn,Koog aan de Zaan,...,Amstelveen,Amsterdam,Amsterdam,Amstelveen,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Uithoorn,Amsterdam
dim_incident_incident_type,Assistentie Politie,Reanimeren,Assistentie Ambulance,Assistentie Ambulance,Persoon te water,Assistentie Ambulance,Assistentie Politie,OMS / automatische melding,Buitenbrand,Reanimeren,...,Buitenbrand,OMS / automatische melding,Buitenbrand,Hulpverlening algemeen,OMS / automatische melding,Reanimeren,Buitenbrand,Buitenbrand,Buitenbrand,Buitenbrand
dim_datum_datum,2017-02-12 00:00:00,2017-02-17 00:00:00,2017-02-18 00:00:00,2017-01-27 00:00:00,2017-01-22 00:00:00,2017-01-20 00:00:00,2017-01-20 00:00:00,2017-01-15 00:00:00,2017-01-13 00:00:00,2017-01-09 00:00:00,...,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-30 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00,2017-12-31 00:00:00
dim_datum_jaar,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,...,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017
dim_datum_maand_nr,2,2,2,1,1,1,1,1,1,1,...,12,12,12,12,12,12,12,12,12,12
dim_datum_maand_dag_nr,12,17,18,27,22,20,20,15,13,9,...,31,31,31,31,31,30,31,31,31,31
dim_datum_week_nr,6,7,7,4,3,3,3,2,2,2,...,52,52,52,52,52,52,52,52,52,52


In [88]:
df_dep.T

,0,1,2,3,4,5,6,7,8,9,...,35843,35844,35845,35846,35847,35848,35849,35850,35851,35852
hub_incident_id,271390,272924,281215,266402,266401,266397,266397,266396,266394,266392,...,295283,295282,295281,295280,295279,295277,295278,295276,298544,298544
inzet_gealarmeerd_datumtijd,2017-03-02 14:04:22,2017-03-23 13:52:57,2017-07-05 20:20:07,2017-01-01 00:01:08,2017-01-01 00:06:09,2017-01-01 00:10:58,2017-01-01 00:11:14,2017-01-01 00:13:08,2017-01-01 00:15:35,2017-01-01 00:17:43,...,2017-12-31 23:27:50,2017-12-31 23:33:16,2017-12-31 23:45:38,2017-12-31 23:47:54,2017-12-31 23:49:49,2017-12-31 23:52:24,2017-12-31 23:52:46,2017-12-31 23:56:38,2017-09-05 16:21:51,2017-09-05 16:21:51
inzet_uitgerukt_datumtijd,NaN,NaN,NaN,2017-01-01 00:01:45,2017-01-01 00:06:38,2017-01-01 00:12:29,NaN,2017-01-01 00:14:01,2017-01-01 00:16:17,NaN,...,2017-12-31 23:29:14,2017-12-31 23:35:08,2017-12-31 23:46:54,NaN,2017-12-31 23:52:55,2017-12-31 23:56:30,2017-12-31 23:55:52,2017-12-31 23:57:09,NaN,2017-09-05 16:23:11
inzet_terplaatse_datumtijd,NaN,NaN,NaN,2017-01-01 00:11:44,2017-01-01 00:11:32,2017-01-01 00:18:35,2017-01-01 00:11:27,2017-01-01 00:17:54,2017-01-01 00:19:55,2017-01-01 00:21:21,...,2017-12-31 23:44:39,2017-12-31 23:37:57,2017-12-31 23:52:23,NaN,2017-12-31 23:57:08,2017-12-31 23:58:43,2017-12-31 23:59:46,2018-01-01 00:02:17,2017-09-05 16:22:03,2017-09-05 16:27:56
inzet_naar_kazerne_datumtijd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
inzet_start_inzet_datumtijd,2017-03-02 14:10:28,2017-03-23 13:52:58,2017-07-05 20:20:07,2017-01-01 00:01:08,2017-01-01 00:06:09,2017-01-01 00:10:58,2017-01-01 00:11:14,2017-01-01 00:13:08,2017-01-01 00:15:35,2017-01-01 00:17:43,...,2017-12-31 23:27:50,2017-12-31 23:33:16,2017-12-31 23:45:38,2017-12-31 23:47:54,2017-12-31 23:49:49,2017-12-31 23:52:24,2017-12-31 23:52:46,2017-12-31 23:56:38,2017-09-05 16:21:51,2017-09-05 16:21:51
inzet_eind_inzet_datumtijd,NaN,NaN,NaN,2017-01-01 00:24:07,2017-01-01 00:13:14,2017-01-01 00:37:52,2017-01-01 00:37:50,2017-01-01 00:24:55,2017-01-01 00:27:57,2017-01-01 00:26:05,...,2018-01-01 00:04:15,2018-01-01 00:07:15,2018-01-01 00:07:29,2018-01-01 00:06:50,2018-01-01 00:02:19,2018-01-01 01:09:36,2018-01-01 00:06:59,2018-01-01 00:43:00,2017-09-05 16:31:18,2017-09-05 16:40:00
inzet_naam_voertuig,Onbekend,Onbekend,Onbekend,133438,133131,133538,13GG02,133332,133331,139232,...,133131,133032,133031,133538,133438,133535,133331,133431,Onbekend,Onbekend
inzet_bijz_voertuig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
inzet_uitrusting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
print("the shape of the incidents dataset is {}".format(df_in.shape))
print("the shape of the deployment dataset is {}".format(df_dep.shape))
print("the time span of the data set incidents is: {}".format((min(df_in['dim_datum_datum']), max(df_in['dim_datum_datum']))))
print(print("the time span of the data set deploy is: {}".format((min(df_dep['inzet_gealarmeerd_datumtijd']), 
                                                                  max(df_dep['inzet_gealarmeerd_datumtijd'])))))

incidents_df_in = list(set(df_in['dim_incident_id'].unique()) - set(df_dep['hub_incident_id'].unique()))
incidents_df_dep = list(set(df_dep['hub_incident_id'].unique()) - set(df_in['dim_incident_id'].unique()))

print('the following id are only at incidents {}'.format(incidents_df_in))
print('the following id are only at deploy {}'.format(incidents_df_dep))

the shape of the incidents dataset is (12754, 57)
the shape of the events dataset is (35853, 55)
the time span of the data set incidents is: ('2017-01-01 00:00:00', '2017-12-31 00:00:00')
the time span of the data set deploy is: ('2017-01-01 00:01:08', '2017-12-31 23:56:38')
None
the following id are only at incidents [301810]
the following id are only at deploy [288769, 272392, 278537, 270346, 292875, 294926, 272399, 274447, 274448, 276498, 278545, 294932, 284687, 292882, 288793, 274458, 282653, 276510, 284704, 268324, 292906, 270380, 274476, 280622, 292908, 268336, 292910, 286770, 280631, 286775, 270393, 290872, 286779, 280636, 270398, 280638, 282687, 276545, 276550, 280646, 282696, 272460, 280655, 288849, 276563, 288854, 288857, 274522, 290906, 290910, 274527, 282720, 272484, 276580, 278629, 292964, 292970, 274542, 284788, 284790, 284791, 290935, 268410, 288890, 288892, 288894, 290945, 280708, 284808, 276617, 282761, 272523, 272524, 284812, 268431, 280721, 286868, 290964, 288918, 27

In [32]:
df_in['Aux'] = 1
names = list(df_in)
# df_in.groupby('Aux')[names].apply(lambda x: x.isnull().sum()).T

In [56]:
df_in.describe().T.dropna()[['mean', 'std', 'min', 'max']]

,mean,std,min,max
dim_incident_id,2.807996e+05,8.522680e+03,266099.0,301810.0
dim_datum_jaar,2.017000e+03,0.000000e+00,2017.0,2017.0
dim_datum_maand_nr,6.515525e+00,3.545112e+00,1.0,12.0
dim_datum_maand_dag_nr,1.581245e+01,8.865167e+00,1.0,31.0
dim_datum_week_nr,2.706531e+01,1.548334e+01,1.0,52.0
dim_datum_kwartaal,2.507370e+00,1.130052e+00,1.0,4.0
dim_prioriteit_prio,1.280069e+00,4.576980e-01,1.0,3.0
dim_tijd_uur,1.316316e+01,6.240115e+00,0.0,23.0
hub_object_id,3.294998e+05,1.991621e+05,142.0,670592.0
sat_incident_classificatie_hub_incident_id,2.807996e+05,8.522680e+03,266099.0,301810.0


In [51]:
df_dep.describe().T.dropna()[['mean', 'std', 'min', 'max']]


,mean,std,min,max
hub_incident_id,2.807515e+05,8.387275e+03,266099.0,298544.0
dim_tijd_uur,1.310392e+01,6.172071e+00,0.0,23.0
dim_datum_jaar,2.017000e+03,0.000000e+00,2017.0,2017.0
dim_datum_maand_nr,6.494101e+00,3.496663e+00,1.0,12.0
dim_datum_maand_dag_nr,1.588712e+01,8.778988e+00,1.0,31.0
dim_datum_week_nr,2.670569e+01,1.525415e+01,1.0,52.0
dim_datum_kwartaal,2.500070e+00,1.120430e+00,1.0,4.0
inzet_terplaatse_volgnummer,5.774133e-01,3.477804e+00,0.0,157.0
dim_inzet_nr,9.235395e+08,1.196074e+07,903567190.0,946647831.0
hub_inzet_id,3.331924e+05,1.167868e+04,312968.0,358203.0


# EDA on the components of the response time

In [ ]:
#(in) incident starts = dim_incident_start_datumtijd
#(in) incident finish = dim_incident_eind_datumtijd
#(dep) alarm is activated = inzet_gealarmeerd_datumtijd
#(dep) leave = inzet_uitgerukt_datumtijd
#(dep) arrive = inzet_terplaatse_datumtijd

In [57]:
print(list(df_in))

['dim_incident_id', 'dim_incident_straatnaam_nen', 'dim_incident_postcode', 'dim_incident_plaatsnaam', 'dim_incident_incident_type', 'dim_datum_datum', 'dim_datum_jaar', 'dim_datum_maand_nr', 'dim_datum_maand_dag_nr', 'dim_datum_week_nr', 'dim_datum_kwartaal', 'dim_datum_maand_naam_nl', 'dim_datum_dag_naam_nl', 'dim_datum_jaar_maand_code', 'dim_prioriteit_prio', 'dim_tijd_uur', 'dim_tijd_dagdeel', 'inc_dim_object_type', 'inc_dim_object_functie', 'inc_dim_object_naam', 'dim_classificatie_object_type', 'dim_classificatie_brand_type', 'dim_incident_afstand', 'dim_incident_start_datumtijd', 'dim_incident_eind_datumtijd', 'key', 'dim_incident_brandoorzaak_hoofd', 'dim_incident_brandoorzaak_sub', 'dim_incident_brandoorzaak_brandschademodel', 'hub_object_id', 'sat_incident_classificatie_ind_risicoprofiel', 'sat_incident_details_start_tijd', 'sat_incident_details_start_datum', 'sat_incident_classificatie_hub_incident_id', 'hub_vak_bk', 'hub_vak_id', 'hub_incident_id', 'sat_kazerne_details_kaz_

In [90]:
def pre_process_data(df_in, df_dep, locations):
    
    inProj  = Proj("+init=EPSG:28992", preserve_units=True)
    outProj = Proj("+init=EPSG:4326") # WGS84 in degrees and not EPSG:3857 in meters)

    def projections(x,y, inProj, outProj):
        """
        in this funtion we transform the data from the corrdinate system to gps
        """
        longitd, latitud = transform(inProj,outProj,x,y)
        return longitd, latitud
    
    
    keep_in = ['dim_incident_id','st_x', 'st_y', 'dim_incident_incident_type', 'inc_dim_object_naam', 
               'dim_incident_start_datumtijd', 'dim_incident_eind_datumtijd', 'dim_prioriteit_prio']
    
    keep_dep = ['hub_incident_id', 'inzet_uitgerukt_datumtijd', 'inzet_terplaatse_datumtijd', 'voertuig_groep',
            'kazerne_groep', 'inzet_kazerne_naam']
    
    df_in = df_in[keep_in]
    df_dep = df_dep[keep_dep]
    
    M = df_dep.merge(df_in, left_on='hub_incident_id', right_on='dim_incident_id', how = 'inner')
    
    M['inzet_kazerne_naam'] = M['inzet_kazerne_naam'].apply(lambda x: x.lower())
    locations['kazerne'] = locations['kazerne'].apply(lambda x: x.lower())
    
    M = locations.merge(M, left_on='kazerne', right_on='inzet_kazerne_naam', how = 'inner')
    return M

M = pre_process_data(df_in, df_dep, locations)
M.head()


,kazerne,adres,postcode,plaats,lon,lat,dispatch (seconds),hub_incident_id,inzet_uitgerukt_datumtijd,inzet_terplaatse_datumtijd,...,kazerne_groep,inzet_kazerne_naam,dim_incident_id,st_x,st_y,dim_incident_incident_type,inc_dim_object_naam,dim_incident_start_datumtijd,dim_incident_eind_datumtijd,dim_prioriteit_prio
0,anton,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0,266343,2017-01-01 01:00:00,2017-01-01 01:03:35,...,Anton,anton,266343,126876,481495,Buitenbrand,groeneveen,2017-01-01 00:56:36,2017-01-01 01:15:01,1
1,anton,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0,266316,2017-01-01 01:29:04,2017-01-01 01:31:37,...,Anton,anton,266316,125552,480660,Buitenbrand,parkeergarage hoogoord,2017-01-01 01:21:05,2017-01-01 01:35:05,2
2,anton,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0,266303,2017-01-01 02:10:35,2017-01-01 02:20:01,...,Anton,anton,266303,129338,484716,Buitenbrand,fritz dietrich kahlenbergstraat,2017-01-01 01:28:58,2017-01-01 02:37:48,2
3,anton,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0,266263,2017-01-01 03:17:31,2017-01-01 03:24:16,...,Anton,anton,266263,125718,481321,Buitenbrand,florijn,2017-01-01 02:22:34,2017-01-01 03:36:55,2
4,anton,Langbroekdreef 10,1108 EB,Amsterdam,4.973266,52.307286,197.0,266209,2017-01-01 05:06:30,2017-01-01 05:11:42,...,Anton,anton,266209,126652,481510,Buitenbrand,gooioord,2017-01-01 04:59:23,2017-01-01 05:22:00,2


data issues:
(a) no all the stations have dispatch time
(b) no all the stations (kazerne) are in the deploy dataset (inzet_kazerne_naam) ['anton', 'dirk', 'hendrik', 'ijsbrand', 'nico', 'osdorp', 'pieter', 'teunis', 'victor', 'willem', 'zebra']



In [102]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

M['haversine_distance (Km)'] = np.vectorize(haversine)(M['lon'], M['lat'], M['st_y'], M['st_x'])/1000
M.T

,0,1,2,3,4,5,6,7,8,9,...,14045,14046,14047,14048,14049,14050,14051,14052,14053,14054
kazerne,anton,anton,anton,anton,anton,anton,anton,anton,anton,anton,...,zebra,zebra,zebra,zebra,zebra,zebra,zebra,zebra,zebra,zebra
adres,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,Langbroekdreef 10,...,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60,Ijdoornlaan 60
postcode,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,1108 EB,...,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV,1024 KV
plaats,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,...,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam,Amsterdam
lon,4.97327,4.97327,4.97327,4.97327,4.97327,4.97327,4.97327,4.97327,4.97327,4.97327,...,4.95483,4.95483,4.95483,4.95483,4.95483,4.95483,4.95483,4.95483,4.95483,4.95483
lat,52.3073,52.3073,52.3073,52.3073,52.3073,52.3073,52.3073,52.3073,52.3073,52.3073,...,52.3962,52.3962,52.3962,52.3962,52.3962,52.3962,52.3962,52.3962,52.3962,52.3962
dispatch (seconds),197,197,197,197,197,197,197,197,197,197,...,193,193,193,193,193,193,193,193,193,193
hub_incident_id,266343,266316,266303,266263,266209,266185,266186,266199,266160,266148,...,294801,294792,295026,295021,294992,294969,294959,294950,294939,295282
inzet_uitgerukt_datumtijd,2017-01-01 01:00:00,2017-01-01 01:29:04,2017-01-01 02:10:35,2017-01-01 03:17:31,2017-01-01 05:06:30,2017-01-01 07:22:32,2017-01-01 08:19:54,2017-01-01 08:52:33,2017-01-01 11:38:24,2017-01-01 14:01:16,...,2017-12-30 21:04:45,2017-12-30 23:33:54,2017-12-30 23:46:42,2017-12-31 01:02:04,2017-12-31 15:01:49,2017-12-31 18:32:18,2017-12-31 18:58:25,NaN,2017-12-31 20:11:30,2017-12-31 23:35:08
inzet_terplaatse_datumtijd,2017-01-01 01:03:35,2017-01-01 01:31:37,2017-01-01 02:20:01,2017-01-01 03:24:16,2017-01-01 05:11:42,2017-01-01 07:24:25,2017-01-01 08:24:27,2017-01-01 08:52:34,2017-01-01 11:39:21,2017-01-01 14:09:47,...,2017-12-30 21:12:57,NaN,2017-12-30 23:48:35,2017-12-31 01:04:17,2017-12-31 15:08:58,NaN,2017-12-31 19:06:31,NaN,2017-12-31 20:14:56,2017-12-31 23:37:57


In [ ]:
# to transform latitud and longitud 